In [1]:
"""
Multi-Agent Phishing Detection System with Kaggle Datasets
Uses DistilBERT models with comprehensive evaluation metrics from research papers
"""

# Install required packages
!pip install transformers datasets torch scikit-learn pandas accelerate kagglehub matplotlib seaborn tabulate -q

import torch
import pandas as pd
import numpy as np
import os
import glob
from transformers import (
    DistilBertTokenizer, 
    DistilBertForSequenceClassification,
    Trainer, 
    TrainingArguments,
    EarlyStoppingCallback
)
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, 
    precision_recall_fscore_support, 
    classification_report,
    confusion_matrix,
    roc_curve,
    auc,
    roc_auc_score
)
import json
import matplotlib.pyplot as plt
import seaborn as sns
import kagglehub
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# ============================================================================
# STEP 1: Download and Load Datasets from Kaggle
# ============================================================================

print("=" * 70)
print("STEP 1: DOWNLOADING DATASETS FROM KAGGLE")
print("=" * 70)

# Download Email Phishing Dataset
print("\n📥 Downloading Email Phishing Dataset...")
email_path = kagglehub.dataset_download("naserabdullahalam/phishing-email-dataset")
print(f"Email dataset path: {email_path}")

# Download URL Phishing Dataset
print("\n📥 Downloading URL Phishing Dataset...")
url_path = kagglehub.dataset_download("shashwatwork/web-page-phishing-detection-dataset")
print(f"URL dataset path: {url_path}")

# ============================================================================
# STEP 2: Load and Process Email Dataset
# ============================================================================

print("\n" + "=" * 70)
print("STEP 2: LOADING AND PROCESSING EMAIL DATASET")
print("=" * 70)

# Find CSV files in email dataset directory
email_files = glob.glob(os.path.join(email_path, "**/*.csv"), recursive=True)
print(f"\nFound {len(email_files)} email files")

# Try to load the main phishing_email.csv first
email_df = None
preferred_files = ['phishing_email.csv', 'CEAS_08.csv']

for preferred in preferred_files:
    for file in email_files:
        if preferred in file:
            try:
                temp_df = pd.read_csv(file)
                print(f"\n✓ Loading {os.path.basename(file)}")
                print(f"  Columns: {temp_df.columns.tolist()}")
                print(f"  Shape: {temp_df.shape}")
                
                # Try to identify text and label columns
                text_col = None
                label_col = None
                
                for col in temp_df.columns:
                    col_lower = col.lower()
                    if 'body' in col_lower or 'email' in col_lower or 'text' in col_lower:
                        text_col = col
                    if 'label' in col_lower or 'class' in col_lower:
                        label_col = col
                
                if text_col and label_col:
                    email_df = temp_df[[text_col, label_col]].copy()
                    email_df.columns = ['text', 'label']
                    print(f"  Using: text='{text_col}', label='{label_col}'")
                    break
            except Exception as e:
                print(f"  Error: {e}")
                continue
    
    if email_df is not None:
        break

if email_df is None:
    raise ValueError("Could not load email dataset. Please check the file structure.")

# Clean email dataset
email_df = email_df.dropna()
email_df['text'] = email_df['text'].astype(str)
email_df['text'] = email_df['text'].str[:1000]  # Limit text length

# Normalize labels to 0 (legitimate) and 1 (phishing)
unique_labels = email_df['label'].unique()
print(f"\n  Unique labels: {unique_labels}")

if email_df['label'].dtype == 'object':
    phishing_keywords = ['phish', 'spam', '1', 'true']
    email_df['label'] = email_df['label'].apply(
        lambda x: 1 if any(key in str(x).lower() for key in phishing_keywords) else 0
    )
else:
    email_df['label'] = email_df['label'].apply(lambda x: 1 if x != 0 else 0)

print(f"\n✓ Email dataset processed:")
print(f"  Total: {len(email_df)} samples")
print(f"  Legitimate: {sum(email_df['label'] == 0)}")
print(f"  Phishing: {sum(email_df['label'] == 1)}")

# ============================================================================
# STEP 3: Load and Process URL Dataset
# ============================================================================

print("\n" + "=" * 70)
print("STEP 3: LOADING AND PROCESSING URL DATASET")
print("=" * 70)

# Find CSV files in URL dataset directory
url_files = glob.glob(os.path.join(url_path, "**/*.csv"), recursive=True)
print(f"\nFound {len(url_files)} URL files")

# Load URL dataset
url_df = None
for file in url_files:
    try:
        temp_df = pd.read_csv(file)
        print(f"\n✓ Loading {os.path.basename(file)}")
        print(f"  Columns: {temp_df.columns.tolist()[:10]}...")  # Show first 10
        print(f"  Shape: {temp_df.shape}")
        
        # Look specifically for 'url' and 'status' columns
        if 'url' in temp_df.columns and 'status' in temp_df.columns:
            url_df = temp_df[['url', 'status']].copy()
            url_df.columns = ['text', 'label']
            print(f"  Using: text='url', label='status'")
            break
        elif 'url' in temp_df.columns:
            # Find any potential label column
            for col in temp_df.columns:
                if col.lower() in ['label', 'class', 'status', 'target']:
                    url_df = temp_df[['url', col]].copy()
                    url_df.columns = ['text', 'label']
                    print(f"  Using: text='url', label='{col}'")
                    break
            if url_df is not None:
                break
                
    except Exception as e:
        print(f"  Error: {e}")
        continue

if url_df is None:
    raise ValueError("Could not load URL dataset. Please check the file structure.")

# Clean URL dataset
url_df = url_df.dropna()
url_df['text'] = url_df['text'].astype(str)
url_df['text'] = url_df['text'].str[:500]  # Limit URL length

# Normalize labels to 0 (legitimate) and 1 (phishing)
unique_labels = url_df['label'].unique()
print(f"\n  Unique labels: {unique_labels}")

if url_df['label'].dtype == 'object':
    phishing_keywords = ['phish', 'bad', 'malicious', '1']
    url_df['label'] = url_df['label'].apply(
        lambda x: 1 if any(key in str(x).lower() for key in phishing_keywords) else 0
    )
else:
    url_df['label'] = url_df['label'].apply(lambda x: 1 if x != 0 else 0)

print(f"\n✓ URL dataset processed:")
print(f"  Total: {len(url_df)} samples")
print(f"  Legitimate: {sum(url_df['label'] == 0)}")
print(f"  Phishing: {sum(url_df['label'] == 1)}")
print(f"\n  Sample URLs:")
for i in range(min(3, len(url_df))):
    print(f"    {url_df.iloc[i]['text'][:60]}... (label: {url_df.iloc[i]['label']})")

# ============================================================================
# STEP 4: Balance Datasets if Needed
# ============================================================================

print("\n" + "=" * 70)
print("STEP 4: BALANCING DATASETS")
print("=" * 70)

def balance_dataset(df, max_samples=5000):
    """Balance dataset by undersampling majority class"""
    legitimate = df[df['label'] == 0]
    phishing = df[df['label'] == 1]
    
    # Determine the minimum count
    min_count = min(len(legitimate), len(phishing), max_samples)
    
    # Sample equally from both classes
    legitimate_balanced = legitimate.sample(n=min_count, random_state=42)
    phishing_balanced = phishing.sample(n=min_count, random_state=42)
    
    # Combine and shuffle
    balanced_df = pd.concat([legitimate_balanced, phishing_balanced])
    balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)
    
    return balanced_df

# Balance email dataset
email_df_balanced = balance_dataset(email_df, max_samples=5000)
print(f"\n✓ Email dataset balanced:")
print(f"  Total: {len(email_df_balanced)} samples")
print(f"  Legitimate: {sum(email_df_balanced['label'] == 0)}")
print(f"  Phishing: {sum(email_df_balanced['label'] == 1)}")

# Balance URL dataset
url_df_balanced = balance_dataset(url_df, max_samples=5000)
print(f"\n✓ URL dataset balanced:")
print(f"  Total: {len(url_df_balanced)} samples")
print(f"  Legitimate: {sum(url_df_balanced['label'] == 0)}")
print(f"  Phishing: {sum(url_df_balanced['label'] == 1)}")

# ============================================================================
# STEP 5: Create Umpire Dataset
# ============================================================================

print("\n" + "=" * 70)
print("STEP 5: CREATING UMPIRE DATASET")
print("=" * 70)

# Create umpire dataset (0=email, 1=url)
email_umpire = email_df_balanced[['text']].copy()
email_umpire['label'] = 0  # Email category

url_umpire = url_df_balanced[['text']].copy()
url_umpire['label'] = 1  # URL category

umpire_df = pd.concat([email_umpire, url_umpire], ignore_index=True)
umpire_df = umpire_df.sample(frac=1, random_state=42).reset_index(drop=True)

print(f"✓ Umpire dataset created: {len(umpire_df)} samples")
print(f"  Email samples: {sum(umpire_df['label'] == 0)}")
print(f"  URL samples: {sum(umpire_df['label'] == 1)}")

# ============================================================================
# STEP 6: Custom Dataset Class
# ============================================================================

class PhishingDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# ============================================================================
# STEP 7: Enhanced Training Function with Comprehensive Metrics
# ============================================================================

def calculate_tpr_at_fpr(y_true, y_scores, fpr_threshold=0.01):
    """Calculate TPR at specific FPR threshold (from URLTran paper)"""
    try:
        fpr, tpr, thresholds = roc_curve(y_true, y_scores)
        idx = np.where(fpr <= fpr_threshold)[0]
        if len(idx) > 0:
            return tpr[idx[-1]]
    except:
        pass
    return 0.0

def plot_roc_curve(y_true, y_scores, model_name, save_path):
    """Plot ROC curve"""
    try:
        fpr, tpr, _ = roc_curve(y_true, y_scores)
        roc_auc = auc(fpr, tpr)
        
        plt.figure(figsize=(10, 8))
        plt.plot(fpr, tpr, color='darkorange', lw=2, 
                 label=f'ROC curve (AUC = {roc_auc:.4f})')
        plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title(f'ROC Curve - {model_name}')
        plt.legend(loc="lower right")
        plt.grid(True, alpha=0.3)
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.close()
        print(f"  ✓ ROC curve saved to {save_path}")
    except Exception as e:
        print(f"  ⚠ Could not plot ROC curve: {e}")

def plot_confusion_matrix(y_true, y_pred, model_name, save_path):
    """Plot confusion matrix"""
    try:
        cm = confusion_matrix(y_true, y_pred)
        
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                    xticklabels=['Legitimate', 'Phishing'],
                    yticklabels=['Legitimate', 'Phishing'])
        plt.title(f'Confusion Matrix - {model_name}')
        plt.ylabel('True Label')
        plt.xlabel('Predicted Label')
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.close()
        print(f"  ✓ Confusion matrix saved to {save_path}")
    except Exception as e:
        print(f"  ⚠ Could not plot confusion matrix: {e}")

def train_model(train_texts, train_labels, val_texts, val_labels, 
                model_name, num_epochs=3, batch_size=16):
    """
    Train DistilBERT model with comprehensive evaluation metrics
    """
    
    print(f"\n{'=' * 70}")
    print(f"Training {model_name}")
    print(f"{'=' * 70}")
    
    # Initialize tokenizer and model
    print(f"\n📥 Loading DistilBERT tokenizer and model...")
    tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
    model = DistilBertForSequenceClassification.from_pretrained(
        'distilbert-base-uncased',
        num_labels=2,
        dropout=0.3,
        attention_dropout=0.3
    )
    
    # Tokenize datasets
    print(f"📝 Tokenizing {len(train_texts)} training samples...")
    train_encodings = tokenizer(
        train_texts,
        truncation=True,
        padding=True,
        max_length=128,
        return_tensors='pt'
    )
    
    print(f"📝 Tokenizing {len(val_texts)} validation samples...")
    val_encodings = tokenizer(
        val_texts,
        truncation=True,
        padding=True,
        max_length=128,
        return_tensors='pt'
    )
    
    # Create datasets
    train_dataset = PhishingDataset(train_encodings, train_labels)
    val_dataset = PhishingDataset(val_encodings, val_labels)
    
    # Compute metrics
    def compute_metrics(pred):
        labels = pred.label_ids
        preds = pred.predictions.argmax(-1)
        precision, recall, f1, _ = precision_recall_fscore_support(
            labels, preds, average='binary', zero_division=0
        )
        acc = accuracy_score(labels, preds)
        return {
            'accuracy': acc,
            'f1': f1,
            'precision': precision,
            'recall': recall
        }
    
    # Training arguments
    training_args = TrainingArguments(
        output_dir=f'./results_{model_name}',
        num_train_epochs=num_epochs,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        warmup_steps=50,
        weight_decay=0.01,
        logging_dir=f'./logs_{model_name}',
        logging_steps=50,
        eval_strategy="steps",
        eval_steps=100,
        save_strategy="steps",
        save_steps=100,
        load_best_model_at_end=True,
        metric_for_best_model="f1",
        greater_is_better=True,
        save_total_limit=1,
        learning_rate=2e-5,
        max_grad_norm=1.0,
        fp16=torch.cuda.is_available(),
        report_to="none",  # Disable wandb
    )
    
    # Initialize trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    )
    
    # Train
    print(f"\n🚀 Starting training for {num_epochs} epochs...")
    trainer.train()
    
    # Detailed evaluation on validation set
    print(f"\n📊 Computing comprehensive metrics...")
    predictions = trainer.predict(val_dataset)
    y_pred = predictions.predictions.argmax(-1)
    y_scores = torch.softmax(torch.tensor(predictions.predictions), dim=1)[:, 1].numpy()
    y_true = val_labels
    
    # Calculate all metrics
    precision, recall, f1, _ = precision_recall_fscore_support(
        y_true, y_pred, average='binary', zero_division=0
    )
    accuracy = accuracy_score(y_true, y_pred)
    
    # ROC-AUC and TPR@FPR metrics (from URLTran paper)
    try:
        roc_auc = roc_auc_score(y_true, y_scores)
    except:
        roc_auc = 0.0
    
    tpr_001 = calculate_tpr_at_fpr(y_true, y_scores, 0.0001)  # 0.01% FPR
    tpr_01 = calculate_tpr_at_fpr(y_true, y_scores, 0.001)    # 0.1% FPR
    tpr_1 = calculate_tpr_at_fpr(y_true, y_scores, 0.01)      # 1% FPR
    
    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    
    # Calculate per-class metrics (from RoBERTa paper)
    precision_per_class, recall_per_class, f1_per_class, support = \
        precision_recall_fscore_support(y_true, y_pred, average=None, zero_division=0)
    
    results = {
        'accuracy': float(accuracy),
        'precision': float(precision),
        'recall': float(recall),
        'f1_score': float(f1),
        'roc_auc': float(roc_auc),
        'tpr@0.01%_fpr': float(tpr_001),
        'tpr@0.1%_fpr': float(tpr_01),
        'tpr@1%_fpr': float(tpr_1),
        'confusion_matrix': {
            'true_negative': int(tn),
            'false_positive': int(fp),
            'false_negative': int(fn),
            'true_positive': int(tp)
        },
        'per_class_metrics': {
            'legitimate': {
                'precision': float(precision_per_class[0]),
                'recall': float(recall_per_class[0]),
                'f1_score': float(f1_per_class[0]),
                'support': int(support[0])
            },
            'phishing': {
                'precision': float(precision_per_class[1]),
                'recall': float(recall_per_class[1]),
                'f1_score': float(f1_per_class[1]),
                'support': int(support[1])
            }
        }
    }
    
    # Print results
    print(f"\n{'=' * 70}")
    print(f"VALIDATION RESULTS - {model_name}")
    print(f"{'=' * 70}")
    print(f"Accuracy:           {accuracy:.4f}")
    print(f"Precision:          {precision:.4f}")
    print(f"Recall:             {recall:.4f}")
    print(f"F1-Score:           {f1:.4f}")
    print(f"ROC-AUC:            {roc_auc:.4f}")
    print(f"\nTPR at Different FPR Thresholds:")
    print(f"  TPR @ 0.01% FPR:  {tpr_001:.4f}")
    print(f"  TPR @ 0.1% FPR:   {tpr_01:.4f}")
    print(f"  TPR @ 1% FPR:     {tpr_1:.4f}")
    print(f"\nConfusion Matrix:")
    print(f"  TN: {tn:6d}  |  FP: {fp:6d}")
    print(f"  FN: {fn:6d}  |  TP: {tp:6d}")
    print(f"\nPer-Class Metrics:")
    print(f"  Legitimate - P: {precision_per_class[0]:.4f}, R: {recall_per_class[0]:.4f}, F1: {f1_per_class[0]:.4f}")
    print(f"  Phishing   - P: {precision_per_class[1]:.4f}, R: {recall_per_class[1]:.4f}, F1: {f1_per_class[1]:.4f}")
    
    # Generate plots
    model_clean_name = model_name.lower().replace(" ", "_")
    plot_roc_curve(y_true, y_scores, model_name, f'{model_clean_name}_roc_curve.png')
    plot_confusion_matrix(y_true, y_pred, model_name, f'{model_clean_name}_confusion_matrix.png')
    
    # Save model
    model_path = f'./{model_clean_name}'
    trainer.save_model(model_path)
    tokenizer.save_pretrained(model_path)
    print(f"\n✓ Model saved to {model_path}")
    
    return model, tokenizer, results

# ============================================================================
# STEP 8: Train All Three Models
# ============================================================================

print("\n" + "=" * 70)
print("STEP 8: TRAINING ALL MODELS")
print("=" * 70)

# Prepare splits for Email Phishing Model
print("\n📋 Preparing Email Phishing Model data...")
email_texts = email_df_balanced['text'].tolist()
email_labels = email_df_balanced['label'].tolist()
email_train_texts, email_val_texts, email_train_labels, email_val_labels = train_test_split(
    email_texts, email_labels, test_size=0.2, random_state=42, stratify=email_labels
)
print(f"  Train: {len(email_train_texts)}, Val: {len(email_val_texts)}")

# Prepare splits for URL Phishing Model
print("\n📋 Preparing URL Phishing Model data...")
url_texts = url_df_balanced['text'].tolist()
url_labels = url_df_balanced['label'].tolist()
url_train_texts, url_val_texts, url_train_labels, url_val_labels = train_test_split(
    url_texts, url_labels, test_size=0.2, random_state=42, stratify=url_labels
)
print(f"  Train: {len(url_train_texts)}, Val: {len(url_val_texts)}")

# Prepare splits for Umpire Model
print("\n📋 Preparing Umpire Model data...")
umpire_texts = umpire_df['text'].tolist()
umpire_labels = umpire_df['label'].tolist()
umpire_train_texts, umpire_val_texts, umpire_train_labels, umpire_val_labels = train_test_split(
    umpire_texts, umpire_labels, test_size=0.2, random_state=42, stratify=umpire_labels
)
print(f"  Train: {len(umpire_train_texts)}, Val: {len(umpire_val_texts)}")

# Train Email Phishing Model
email_model, email_tokenizer, email_results = train_model(
    email_train_texts, email_train_labels,
    email_val_texts, email_val_labels,
    "Email_Phishing_Model",
    num_epochs=3,
    batch_size=16
)

# Train URL Phishing Model
url_model, url_tokenizer, url_results = train_model(
    url_train_texts, url_train_labels,
    url_val_texts, url_val_labels,
    "URL_Phishing_Model",
    num_epochs=3,
    batch_size=16
)

# Train Umpire Model
umpire_model, umpire_tokenizer, umpire_results = train_model(
    umpire_train_texts, umpire_train_labels,
    umpire_val_texts, umpire_val_labels,
    "Umpire_Model",
    num_epochs=3,
    batch_size=16
)

# ============================================================================
# STEP 9: Comparative Analysis and Summary
# ============================================================================

print("\n" + "=" * 70)
print("COMPREHENSIVE PERFORMANCE SUMMARY")
print("=" * 70)

summary = {
    "email_model": {
        "dataset_size": len(email_df_balanced),
        "train_samples": len(email_train_texts),
        "val_samples": len(email_val_texts),
        "metrics": email_results
    },
    "url_model": {
        "dataset_size": len(url_df_balanced),
        "train_samples": len(url_train_texts),
        "val_samples": len(url_val_texts),
        "metrics": url_results
    },
    "umpire_model": {
        "dataset_size": len(umpire_df),
        "train_samples": len(umpire_train_texts),
        "val_samples": len(umpire_val_texts),
        "metrics": umpire_results
    }
}

# Save comprehensive summary
with open('comprehensive_training_summary.json', 'w') as f:
    json.dump(summary, f, indent=2)

# Create comparison table
print("\n" + "=" * 70)
print("MODEL COMPARISON TABLE")
print("=" * 70)
print(f"\n{'Metric':<25} {'Email Model':<15} {'URL Model':<15} {'Umpire Model':<15}")
print("-" * 70)

metrics_to_compare = [
    ('Accuracy', 'accuracy'),
    ('Precision', 'precision'),
    ('Recall', 'recall'),
    ('F1-Score', 'f1_score'),
    ('ROC-AUC', 'roc_auc'),
    ('TPR @ 0.01% FPR', 'tpr@0.01%_fpr'),
    ('TPR @ 0.1% FPR', 'tpr@0.1%_fpr'),
    ('TPR @ 1% FPR', 'tpr@1%_fpr'),
]

for metric_name, metric_key in metrics_to_compare:
    email_val = email_results.get(metric_key, 0)
    url_val = url_results.get(metric_key, 0)
    umpire_val = umpire_results.get(metric_key, 0)
    print(f"{metric_name:<25} {email_val:<15.4f} {url_val:<15.4f} {umpire_val:<15.4f}")

# Baseline comparison (based on paper benchmarks)
print("\n" + "=" * 70)
print("BASELINE COMPARISON")
print("=" * 70)
print("\nBased on research papers:")
print("  RoBERTa (Email):     Accuracy: 98.45%, F1: 0.98")
print("  URLTran (URL):       Accuracy: 99.67%, TPR@0.01%FPR: 86.80%")
print("  URLNet (Baseline):   Accuracy: 99.45%, TPR@0.01%FPR: 71.20%")
print("  Texception:          Accuracy: 99.66%, TPR@0.01%FPR: 52.15%")

print("\n" + "=" * 70)
print("✅ TRAINING COMPLETED SUCCESSFULLY!")
print("=" * 70)
print("\n📦 Saved artifacts:")
print("  1. ./email_phishing_model/")
print("  2. ./url_phishing_model/")
print("  3. ./umpire_model/")
print("  4. comprehensive_training_summary.json")
print("  5. ROC curves and confusion matrices (PNG files)")
print("\n🧪 Ready for testing! Use the testing code to evaluate.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 73.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 44.1 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 57.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 37.5 MB/s eta 0:00:00:00:0100:01
ERROR: pip's d

2025-11-09 19:36:24.585555: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762716984.783492      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762716984.839148      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


📝 Tokenizing 16000 training samples...
📝 Tokenizing 4000 validation samples...

🚀 Starting training for 3 epochs...


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
100,0.004800,0.000299,0.999750,0.999750,0.999500,1.000000
200,0.000000,0.000132,1.000000,1.000000,1.000000,1.000000
300,0.000000,0.000126,1.000000,1.000000,1.000000,1.000000
400,0.000000,0.000114,1.000000,1.000000,1.000000,1.000000
500,0.000000,0.000104,1.000000,1.000000,1.000000,1.000000



📊 Computing comprehensive metrics...



VALIDATION RESULTS - Umpire_Model
Accuracy:           1.0000
Precision:          1.0000
Recall:             1.0000
F1-Score:           1.0000
ROC-AUC:            1.0000

TPR at Different FPR Thresholds:
  TPR @ 0.01% FPR:  1.0000
  TPR @ 0.1% FPR:   1.0000
  TPR @ 1% FPR:     1.0000

Confusion Matrix:
  TN:   2000  |  FP:      0
  FN:      0  |  TP:   2000

Per-Class Metrics:
  Legitimate - P: 1.0000, R: 1.0000, F1: 1.0000
  Phishing   - P: 1.0000, R: 1.0000, F1: 1.0000
  ✓ ROC curve saved to umpire_model_roc_curve.png
  ✓ Confusion matrix saved to umpire_model_confusion_matrix.png

✓ Model saved to ./umpire_model

COMPREHENSIVE PERFORMANCE SUMMARY

MODEL COMPARISON TABLE

Metric                    Email Model     URL Model       Umpire Model   
----------------------------------------------------------------------
Accuracy                  0.9705          0.9265          1.0000         
Precision                 0.9654          0.9466          1.0000         
Recall                  

In [2]:
"""
Testing Code for Multi-Agent Phishing Detection System
Comprehensive evaluation with metrics from research papers
"""

import torch
import pandas as pd
import numpy as np
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import json
from sklearn.metrics import (
    accuracy_score, 
    classification_report, 
    confusion_matrix,
    precision_recall_fscore_support,
    roc_curve,
    auc,
    roc_auc_score
)
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# STEP 1: Load All Three Models
# ============================================================================

print("=" * 70)
print("LOADING TRAINED MODELS")
print("=" * 70)

try:
    # Load Umpire Model
    print("\n📥 Loading Umpire Model...")
    umpire_tokenizer = DistilBertTokenizer.from_pretrained('./umpire_model')
    umpire_model = DistilBertForSequenceClassification.from_pretrained('./umpire_model')
    umpire_model.eval()
    print("✓ Umpire model loaded")

    # Load Email Phishing Model
    print("\n📥 Loading Email Phishing Model...")
    email_tokenizer = DistilBertTokenizer.from_pretrained('./email_phishing_model')
    email_model = DistilBertForSequenceClassification.from_pretrained('./email_phishing_model')
    email_model.eval()
    print("✓ Email phishing model loaded")

    # Load URL Phishing Model
    print("\n📥 Loading URL Phishing Model...")
    url_tokenizer = DistilBertTokenizer.from_pretrained('./url_phishing_model')
    url_model = DistilBertForSequenceClassification.from_pretrained('./url_phishing_model')
    url_model.eval()
    print("✓ URL phishing model loaded")
except Exception as e:
    print(f"\n❌ Error loading models: {e}")
    print("\nPlease ensure you've run the training code first!")
    exit(1)

# ============================================================================
# STEP 2: Define Comprehensive Metrics Functions
# ============================================================================

def calculate_tpr_at_fpr(y_true, y_scores, fpr_threshold=0.01):
    """Calculate TPR at specific FPR threshold (URLTran paper metric)"""
    try:
        fpr, tpr, thresholds = roc_curve(y_true, y_scores)
        idx = np.where(fpr <= fpr_threshold)[0]
        if len(idx) > 0:
            return tpr[idx[-1]]
    except:
        pass
    return 0.0

def calculate_comprehensive_metrics(y_true, y_pred, y_scores):
    """Calculate all metrics from both research papers"""
    
    # Basic metrics (RoBERTa paper)
    precision, recall, f1, _ = precision_recall_fscore_support(
        y_true, y_pred, average='binary', zero_division=0
    )
    accuracy = accuracy_score(y_true, y_pred)
    
    # Per-class metrics
    precision_per_class, recall_per_class, f1_per_class, support = \
        precision_recall_fscore_support(y_true, y_pred, average=None, zero_division=0)
    
    # ROC-AUC metrics (URLTran paper)
    try:
        roc_auc = roc_auc_score(y_true, y_scores)
    except:
        roc_auc = 0.0
    
    # TPR at various FPR thresholds (URLTran paper)
    tpr_0001 = calculate_tpr_at_fpr(y_true, y_scores, 0.0001)  # 0.01% FPR
    tpr_001 = calculate_tpr_at_fpr(y_true, y_scores, 0.001)    # 0.1% FPR
    tpr_01 = calculate_tpr_at_fpr(y_true, y_scores, 0.01)      # 1% FPR
    
    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'roc_auc': roc_auc,
        'tpr@0.01%_fpr': tpr_0001,
        'tpr@0.1%_fpr': tpr_001,
        'tpr@1%_fpr': tpr_01,
        'confusion_matrix': {
            'tn': tn, 'fp': fp, 'fn': fn, 'tp': tp
        },
        'per_class': {
            'legitimate': {
                'precision': precision_per_class[0],
                'recall': recall_per_class[0],
                'f1': f1_per_class[0],
                'support': int(support[0])
            },
            'phishing': {
                'precision': precision_per_class[1],
                'recall': recall_per_class[1],
                'f1': f1_per_class[1],
                'support': int(support[1])
            }
        }
    }

# ============================================================================
# STEP 3: Define Prediction Functions
# ============================================================================

def predict_single(text, tokenizer, model, device='cpu'):
    """Make prediction for a single text input with confidence scores"""
    model.to(device)
    model.eval()
    
    inputs = tokenizer(
        text,
        truncation=True,
        padding=True,
        max_length=128,
        return_tensors='pt'
    ).to(device)
    
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probabilities = torch.softmax(logits, dim=-1)
        prediction = torch.argmax(probabilities, dim=-1).item()
        confidence = probabilities[0][prediction].item()
    
    return prediction, confidence, probabilities[0].cpu().numpy()

def predict_batch(texts, tokenizer, model, device='cpu', batch_size=32):
    """Batch prediction for efficiency"""
    model.to(device)
    model.eval()
    
    all_predictions = []
    all_scores = []
    
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        
        inputs = tokenizer(
            batch_texts,
            truncation=True,
            padding=True,
            max_length=128,
            return_tensors='pt'
        ).to(device)
        
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
            probabilities = torch.softmax(logits, dim=-1)
            predictions = torch.argmax(probabilities, dim=-1)
            
            all_predictions.extend(predictions.cpu().numpy())
            all_scores.extend(probabilities[:, 1].cpu().numpy())
    
    return np.array(all_predictions), np.array(all_scores)

def multi_agent_predict(text, verbose=True):
    """
    Complete multi-agent prediction pipeline with detailed metrics
    """
    
    # Step 1: Umpire routing decision
    umpire_pred, umpire_conf, umpire_probs = predict_single(text, umpire_tokenizer, umpire_model)
    
    request_type = "EMAIL" if umpire_pred == 0 else "URL"
    
    if verbose:
        print(f"\n{'=' * 70}")
        print(f"🎯 UMPIRE DECISION: {request_type}")
        print(f"   Confidence: {umpire_conf:.4f}")
        print(f"   Probabilities - Email: {umpire_probs[0]:.4f}, URL: {umpire_probs[1]:.4f}")
    
    # Step 2: Route to specialist model
    if umpire_pred == 0:  # Email
        specialist_pred, specialist_conf, specialist_probs = predict_single(
            text, email_tokenizer, email_model
        )
        specialist_name = "Email Phishing Model"
    else:  # URL
        specialist_pred, specialist_conf, specialist_probs = predict_single(
            text, url_tokenizer, url_model
        )
        specialist_name = "URL Phishing Model"
    
    # Step 3: Final decision
    final_verdict = "PHISHING" if specialist_pred == 1 else "LEGITIMATE"
    
    if verbose:
        print(f"\n🔍 {specialist_name.upper()}")
        print(f"   Verdict: {final_verdict}")
        print(f"   Confidence: {specialist_conf:.4f}")
        print(f"   Probabilities - Legitimate: {specialist_probs[0]:.4f}, Phishing: {specialist_probs[1]:.4f}")
        print(f"{'=' * 70}\n")
    
    return {
        'text': text[:100] + '...' if len(text) > 100 else text,
        'umpire_decision': request_type,
        'umpire_confidence': umpire_conf,
        'umpire_prediction': umpire_pred,
        'specialist_model': specialist_name,
        'final_verdict': final_verdict,
        'specialist_confidence': specialist_conf,
        'specialist_prediction': specialist_pred,
        'specialist_score': specialist_probs[1]
    }

# ============================================================================
# STEP 4: Predefined Examples Test
# ============================================================================

def test_examples():
    """Test with predefined examples"""
    print("\n" + "=" * 70)
    print("TESTING WITH PREDEFINED EXAMPLES")
    print("=" * 70)
    
    examples = [
        {
            'text': 'Congratulations! You have won $1,000,000 in the Microsoft Lottery! Click here to claim your prize now before it expires! http://win-prize-now.com/claim',
            'type': 'email',
            'expected': 'phishing'
        },
        {
            'text': 'http://paypal-security-update.suspicious-domain.com/verify-account.php?id=12345',
            'type': 'url',
            'expected': 'phishing'
        },
        {
            'text': 'Your meeting with the team has been scheduled for tomorrow at 2:00 PM in Conference Room B. Please confirm your attendance by replying to this email.',
            'type': 'email',
            'expected': 'legitimate'
        },
        {
            'text': 'https://www.google.com/search?q=machine+learning',
            'type': 'url',
            'expected': 'legitimate'
        },
        {
            'text': 'URGENT ACTION REQUIRED: Your account will be suspended in 24 hours unless you verify your identity immediately. Click here: http://secure-bank-verify.ru/login',
            'type': 'email',
            'expected': 'phishing'
        },
        {
            'text': 'https://github.com/pytorch/pytorch',
            'type': 'url',
            'expected': 'legitimate'
        },
        {
            'text': 'Dear customer, we detected unusual activity on your account. Please update your payment information to avoid service interruption.',
            'type': 'email',
            'expected': 'phishing'
        },
        {
            'text': 'http://www.microsoft.com/en-us/security',
            'type': 'url',
            'expected': 'legitimate'
        }
    ]
    
    correct_predictions = 0
    total_predictions = len(examples)
    
    for i, example in enumerate(examples, 1):
        print(f"\n{'─' * 70}")
        print(f"Example {i}/{total_predictions}: {example['type'].upper()}")
        print(f"Expected: {example['expected'].upper()}")
        print(f"Text: {example['text'][:80]}...")
        
        result = multi_agent_predict(example['text'], verbose=True)
        
        # Check if prediction matches expected
        predicted_label = result['final_verdict'].lower()
        if predicted_label == example['expected']:
            correct_predictions += 1
            print("✅ CORRECT PREDICTION")
        else:
            print("❌ INCORRECT PREDICTION")
    
    # Summary
    accuracy = correct_predictions / total_predictions
    print(f"\n{'=' * 70}")
    print(f"PREDEFINED EXAMPLES SUMMARY")
    print(f"{'=' * 70}")
    print(f"Correct Predictions: {correct_predictions}/{total_predictions}")
    print(f"Accuracy: {accuracy:.2%}")

# ============================================================================
# STEP 5: Interactive Testing
# ============================================================================

def interactive_test():
    """Interactive testing mode"""
    print("\n" + "=" * 70)
    print("INTERACTIVE TESTING MODE")
    print("=" * 70)
    print("\nEnter text to test (or 'quit' to exit)")
    print("You can test emails or URLs\n")
    
    while True:
        try:
            text = input("📝 Enter text: ").strip()
            
            if text.lower() == 'quit':
                print("\n👋 Exiting interactive mode...")
                break
            
            if not text:
                print("⚠️  Please enter some text!\n")
                continue
            
            result = multi_agent_predict(text, verbose=True)
            
        except KeyboardInterrupt:
            print("\n\n👋 Exiting interactive mode...")
            break
        except Exception as e:
            print(f"\n⚠️  Error: {e}\n")

# ============================================================================
# STEP 6: Generate Comparison Table with Baselines
# ============================================================================

def generate_comparison_table():
    """Generate comparison table with baseline models from papers"""
    
    print("\n" + "=" * 70)
    print("COMPARISON WITH BASELINE MODELS FROM RESEARCH PAPERS")
    print("=" * 70)
    
    # Load our results
    try:
        with open('comprehensive_training_summary.json', 'r') as f:
            summary = json.load(f)
        
        email_metrics = summary['email_model']['metrics']
        url_metrics = summary['url_model']['metrics']
        umpire_metrics = summary['umpire_model']['metrics']
    except:
        print("\n⚠️  Could not load training summary. Run training first!")
        return
    
    # Define baseline data from papers
    print("\n📊 Performance Comparison:")
    print("\n" + "-" * 70)
    print("EMAIL PHISHING DETECTION")
    print("-" * 70)
    print(f"{'Model':<25} {'Accuracy':<12} {'Precision':<12} {'Recall':<12} {'F1-Score':<12}")
    print("-" * 70)
    print(f"{'RoBERTa (Paper)':<25} {0.9845:<12.4f} {0.97:<12.4f} {1.00:<12.4f} {0.98:<12.4f}")
    print(f"{'Our Email Model':<25} {email_metrics['accuracy']:<12.4f} {email_metrics['precision']:<12.4f} {email_metrics['recall']:<12.4f} {email_metrics['f1_score']:<12.4f}")
    
    print("\n" + "-" * 70)
    print("URL PHISHING DETECTION")
    print("-" * 70)
    print(f"{'Model':<25} {'Accuracy':<12} {'F1-Score':<12} {'TPR@0.01%FPR':<15}")
    print("-" * 70)
    print(f"{'URLTran_BERT (Paper)':<25} {0.9967:<12.4f} {0.9971:<12.4f} {0.8680:<15.4f}")
    print(f"{'URLNet (Paper)':<25} {0.9945:<12.4f} {0.9954:<12.4f} {0.7120:<15.4f}")
    print(f"{'Texception (Paper)':<25} {0.9966:<12.4f} {0.9969:<12.4f} {0.5215:<15.4f}")
    print(f"{'Our URL Model':<25} {url_metrics['accuracy']:<12.4f} {url_metrics['f1_score']:<12.4f} {url_metrics['tpr@0.01%_fpr']:<15.4f}")
    
    print("\n" + "-" * 70)
    print("UMPIRE (ROUTING) MODEL")
    print("-" * 70)
    print(f"Accuracy: {umpire_metrics['accuracy']:.4f}")
    print(f"F1-Score: {umpire_metrics['f1_score']:.4f}")
    
    # Performance summary
    print("\n" + "=" * 70)
    print("PERFORMANCE ANALYSIS")
    print("=" * 70)
    
    print(f"\n✅ Email Model:")
    print(f"   Accuracy: {email_metrics['accuracy']:.4f}")
    print(f"   Status: {'Excellent' if email_metrics['accuracy'] > 0.95 else 'Good' if email_metrics['accuracy'] > 0.90 else 'Needs Improvement'}")
    
    print(f"\n✅ URL Model:")
    print(f"   Accuracy: {url_metrics['accuracy']:.4f}")
    print(f"   TPR@0.01%FPR: {url_metrics['tpr@0.01%_fpr']:.4f}")
    print(f"   Status: {'Excellent' if url_metrics['accuracy'] > 0.95 else 'Good' if url_metrics['accuracy'] > 0.90 else 'Needs Improvement'}")
    
    print(f"\n✅ Umpire Model:")
    print(f"   Routing Accuracy: {umpire_metrics['accuracy']:.4f}")
    print(f"   Status: {'Excellent' if umpire_metrics['accuracy'] > 0.95 else 'Good' if umpire_metrics['accuracy'] > 0.90 else 'Needs Improvement'}")

# ============================================================================
# STEP 7: Main Testing Menu
# ============================================================================

def main_menu():
    """Main testing menu with all options"""
    print("\n" + "=" * 70)
    print("MULTI-AGENT PHISHING DETECTION SYSTEM - COMPREHENSIVE TESTING")
    print("=" * 70)
    
    while True:
        print("\n📋 Select testing mode:")
        print("  1. Test with predefined examples")
        print("  2. Interactive testing (manual input)")
        print("  3. View performance comparison with baselines")
        print("  4. Run all tests")
        print("  5. Exit")
        
        try:
            choice = input("\n👉 Enter your choice (1-5): ").strip()
            
            if choice == '1':
                test_examples()
            
            elif choice == '2':
                interactive_test()
            
            elif choice == '3':
                generate_comparison_table()
            
            elif choice == '4':
                print("\n🔄 Running all tests...\n")
                test_examples()
                generate_comparison_table()
                print("\n✅ All tests completed!")
            
            elif choice == '5':
                print("\n👋 Goodbye!")
                break
            
            else:
                print("⚠️  Invalid choice! Please enter 1-5.")
        
        except KeyboardInterrupt:
            print("\n\n👋 Goodbye!")
            break
        except Exception as e:
            print(f"\n⚠️  Error: {e}")

# ============================================================================
# RUN TESTING
# ============================================================================

if __name__ == "__main__":
    print("\n🚀 Starting Multi-Agent Phishing Detection System Testing")
    print("=" * 70)
    
    # Run predefined examples first
    test_examples()
    
    # Show comparison with baselines
    generate_comparison_table()
    
    # Start interactive menu
    print("\n\n🎯 Ready for additional testing!")
    main_menu()

LOADING TRAINED MODELS

📥 Loading Umpire Model...
✓ Umpire model loaded

📥 Loading Email Phishing Model...
✓ Email phishing model loaded

📥 Loading URL Phishing Model...
✓ URL phishing model loaded

🚀 Starting Multi-Agent Phishing Detection System Testing

TESTING WITH PREDEFINED EXAMPLES

──────────────────────────────────────────────────────────────────────
Example 1/8: EMAIL
Expected: PHISHING
Text: Congratulations! You have won $1,000,000 in the Microsoft Lottery! Click here to...

🎯 UMPIRE DECISION: URL
   Confidence: 1.0000
   Probabilities - Email: 0.0000, URL: 1.0000

🔍 URL PHISHING MODEL
   Verdict: LEGITIMATE
   Confidence: 0.8192
   Probabilities - Legitimate: 0.8192, Phishing: 0.1808

❌ INCORRECT PREDICTION

──────────────────────────────────────────────────────────────────────
Example 2/8: URL
Expected: PHISHING
Text: http://paypal-security-update.suspicious-domain.com/verify-account.php?id=12345...

🎯 UMPIRE DECISION: URL
   Confidence: 1.0000
   Probabilities - Email: 0.


👉 Enter your choice (1-5):  1



TESTING WITH PREDEFINED EXAMPLES

──────────────────────────────────────────────────────────────────────
Example 1/8: EMAIL
Expected: PHISHING
Text: Congratulations! You have won $1,000,000 in the Microsoft Lottery! Click here to...

🎯 UMPIRE DECISION: URL
   Confidence: 1.0000
   Probabilities - Email: 0.0000, URL: 1.0000

🔍 URL PHISHING MODEL
   Verdict: LEGITIMATE
   Confidence: 0.8192
   Probabilities - Legitimate: 0.8192, Phishing: 0.1808

❌ INCORRECT PREDICTION

──────────────────────────────────────────────────────────────────────
Example 2/8: URL
Expected: PHISHING
Text: http://paypal-security-update.suspicious-domain.com/verify-account.php?id=12345...

🎯 UMPIRE DECISION: URL
   Confidence: 1.0000
   Probabilities - Email: 0.0000, URL: 1.0000

🔍 URL PHISHING MODEL
   Verdict: PHISHING
   Confidence: 0.9824
   Probabilities - Legitimate: 0.0176, Phishing: 0.9824

✅ CORRECT PREDICTION

──────────────────────────────────────────────────────────────────────
Example 3/8: EMAIL
Expe


👉 Enter your choice (1-5):  3



COMPARISON WITH BASELINE MODELS FROM RESEARCH PAPERS

📊 Performance Comparison:

----------------------------------------------------------------------
EMAIL PHISHING DETECTION
----------------------------------------------------------------------
Model                     Accuracy     Precision    Recall       F1-Score    
----------------------------------------------------------------------
RoBERTa (Paper)           0.9845       0.9700       1.0000       0.9800      
Our Email Model           0.9705       0.9654       0.9760       0.9707      

----------------------------------------------------------------------
URL PHISHING DETECTION
----------------------------------------------------------------------
Model                     Accuracy     F1-Score     TPR@0.01%FPR   
----------------------------------------------------------------------
URLTran_BERT (Paper)      0.9967       0.9971       0.8680         
URLNet (Paper)            0.9945       0.9954       0.7120         
Texce


👉 Enter your choice (1-5):  5



👋 Goodbye!
